In [3]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

# Simulation Function

In [5]:
def sim(n=2, T=1000, alpha = 0.9, beta = 0.2, gamma = 0.1, lamb0=1, lamb1=5):
    #Defining the three possible values of C
    ck = np.arange(3)   
    
    #Gamma matrix for assigning P(C_t | C_{t-1})
    Gamma = np.array([[1-gamma, 0, gamma], 
                      [0, 1-gamma, gamma], 
                      [beta/2, beta/2, 1-beta]])
    
    #Creating random variables with probabilities based of the gamma matrix
    C_transition = [
        stats.rv_discrete(values=(ck,Gamma[0,])),#P(C_t |C_{t-1} = 0)
        stats.rv_discrete(values=(ck,Gamma[1,])),#P(C_t |C_{t-1} = 1)
        stats.rv_discrete(values=(ck,Gamma[2,])),#P(C_t |C_{t-1} = 2)
    ]

    #Creating output vector of C's
    C = np.zeros(T, np.int64)
    #Initializing the C vector
    C[0] = 2

    #Filling up the C-vector with values
    for i in range(T-1):
        C[i+1] = C_transition[C[i]].rvs()
    
    #CPT of Z
    Z_given_C = np.array([1-alpha, alpha, 0.5])  #P(Z = 1| C =c)
    
    #Initializing Z. size=[n,T] specifies we need to create n copies of a series of T simulations
    Z = stats.bernoulli(Z_given_C[C]).rvs(size=[n,T])
    #input:
    '''
    [C_1, C_2, ... C_T]
    '''
    #output: 
    '''[   
        [Z_11, Z_21, Z_31, ... , Z_T1]
        [Z_12, Z_22, Z_32, ... , Z_T2]
        ...
        [Z_1n, Z_2n, Z_3n, ... , Z_Tn]
    ]'''

    #Initialize X   
    X = stats.poisson(np.where(Z, lamb1, lamb0)).rvs()
    #input
    '''[   
        [f(Z_11), f(Z_21), f(Z_31), ... , f(Z_T1)]
        [f(Z_12), f(Z_22), f(Z_32), ... , f(Z_T2)]
        ...
        [f(Z_1n), f(Z_2n), f(Z_3n), ... , f(Z_Tn)]
        ]    
        where
        f(z) = lamb0+(lamb1-lamb0)*z
    '''

    #output
    '''[   
        [X_11, X_21, X_31, ... , X_T1]
        [X_12, X_22, X_32, ... , X_T2]
        ...
        [X_1n, X_2n, X_3n, ... , X_Tn]
    ]'''

    return C,Z,X

C, Z, X = sim(n = 2,T=10)

In [ ]:
def Clique_Beliefs(Xs, alpha = 0.9, beta = 0.2, lambda_p = [1, 5], gamma = 0.1):

    Xs = Xs.astype(int)
    n = Xs.shape[1]
    T = Xs.shape[0]

    # Defining clique potentials

    psi_L_i = np.array([[1- gamma, 0, gamma], [0, 1-gamma, gamma], [beta/2, beta/2, 1- beta]])             # (C_t, C_{t+1})
    psi_L_1 = np.array([psi_L_i[0] * 0, psi_L_i[1] * 0, psi_L_i[2] * 1])                                         # (C_1,C_2)
    psi_M = np.array([[alpha, 1-alpha], [1-alpha, alpha], [0.5,0.5]])                                      # (C_t, Z_{ti})
    def psi_N(x):                                                                                          # (Z_{ti}, X_{ti})
        def func(z):
            return np.exp(-lambda_p[z]) * lambda_p[z] ** x / math.factorial(x)
        return func
    
    # Forward pass messages
    # Message from N_ti clique to M_ti clique

    delta_Nti_Mti_list = np.empty(shape = (T,n,2))   # (t,i,Z_ti)
    for t in range(T):
        for i in range(n):
            for z_val in [0,1]:
                # The X's are fixed
                delta_Nti_Mti_list[t,i,z_val] = psi_N(Xs[t,i])(z_val) 
            # Renormalizing in order to avoid underflow
            delta_Nti_Mti_list[t, i, :] /= max(delta_Nti_Mti_list[t, i, :])

    # Message from M_ti clique to L_t clique
    
    delta_Mti_Lt_list = np.empty(shape = (T,n,3)) #(t,i,C_t) 
    for t in range(T):
        for i in range(n):
            for c_val in range(3):
                # Summing out Z_{ti}, leaving a factor of C_t
                delta_Mti_Lt_list[t, i, c_val] = sum([psi_M[c_val,z] * delta_Nti_Mti_list[t,i,z] for z in [0, 1]])   
            # Renormalizing
            delta_Mti_Lt_list[t, i, :] /= max(delta_Mti_Lt_list[t, i, :])

    # Message from L_t to L_{t+1} clique

    #Initializing        
    delta_L_forward_list = np.empty(shape = (T-2,3)) #(t,C_{t+1}) t=1,...,T-1

    # Summing out C_{t}, leaving a factor of C_{t+1}
    delta_L_forward_list[0,:] = [sum([psi_L_1[c_i,c_i_] * np.prod(delta_Mti_Lt_list[0,:,c_i], axis = 0) 
                                  for c_i in [0,1,2]]) for c_i_ in [0,1,2]]  

    # Renormalizing
    delta_L_forward_list[0,:] /= max(delta_L_forward_list[0,:])

    for t in range(1,T-2):
        for c_i_ in range(3):
            # Summing out C_{t}, leaving a factor of C_{t+1}
            delta_L_forward_list[t, c_i_] = sum([psi_L_i[c_i,c_i_] * np.prod(delta_Mti_Lt_list[t,:,c_i], axis = 0)
                                             * delta_L_forward_list[t-1,c_i] for c_i in [0,1,2]])   
        # Renormalizing
        delta_L_forward_list[t,:] /= max(delta_L_forward_list[t,:])

    # Clique Beliefs and Downward pass
        
    belief_L_T_minus_1 = np.empty(shape = (3,3))   # (C_{T-1}, C_{T})

    # We loop through all $C_{T-1}$
    for c_val in [0,1,2]:

        # The clique potential
        belief_L_T_minus_1[c_val,:] = psi_L_i[c_val,:] * delta_L_forward_list[T-3,c_val] * np.prod(delta_Mti_Lt_list[T-2,:,:], axis = 0)[c_val] * np.prod(delta_Mti_Lt_list[T-1,:,:], axis = 0)

    # Normalizing
    belief_L_T_minus_1 /= sum(sum(belief_L_T_minus_1))  

    # Initializing beliefs (t,C_t,C_{t+1})
    beliefs_L_t = np.empty(shape = (T-1,3,3))  

    # Defining first belief                                    
    beliefs_L_t[T-2,:,:] = belief_L_T_minus_1 

    # Initializing messages t=1,...,T-1
    delta_L_backward = np.empty(shape = (T-2,3))   #(t, C_{t}) 

    # Compute delta_L_{t+1} to L_t
    for t in range(T-3,-1,-1):                                                       
        for c_val in range(3):     
            # Summing out C_{t+1}                                                 
            delta_L_backward[t,c_val] =  sum([beliefs_L_t[t+1,c_val,c] / delta_L_forward_list[t,c_val] for c in [0,1,2]]) 

        # Renormalizing        
        delta_L_backward[t,:] /= sum(delta_L_backward[t,:])    

        for c_val in range(3):
            # Calculating beliefs
            beliefs_L_t[t, c_val,:] = psi_L_i[c_val,:] * np.prod(delta_Mti_Lt_list[t,:,:], axis = 0)[c_val] * delta_L_backward[t,:]  

        # Normalizing beliefs
        beliefs_L_t[t,:,:] /= np.sum(np.sum(beliefs_L_t[t,:,:], axis = 1), axis = 0)

    delta_Lt_Mti_list = np.empty(shape = (T,n,3)) #(t,n, C_t) t=1,...,T
    for t in range(T-1):
            for i in range(n):
                for c_val in range(3):
                    # Summing out C_{t+1}
                    delta_Lt_Mti_list[t, i, c_val] = sum([beliefs_L_t[t, c_val, c_plus] / delta_Mti_Lt_list[t, i , c_val]  for c_plus in [0,1,2]])

                # Renormalizing
                delta_Lt_Mti_list[t,i,:] /= max(delta_Lt_Mti_list[t,i,:]) 

    # The last clique is special
                
    for i in range(n): 
        for c_plus in range(3):
            # Summing out C_{T-1}
            delta_Lt_Mti_list[T-1, i, c_plus] = sum([beliefs_L_t[T-2, c_val, c_plus] / delta_Mti_Lt_list[T-1 , i, c_plus] for c_val in [0,1,2]])
            
        # Renormalizing
        delta_Lt_Mti_list[T-1,i,:] /= sum(delta_Lt_Mti_list[T-1,i,:]) 

    # Initializing beliefs (t,i,C_t,Z_{ti})
    beliefs_Mti = np.empty(shape = (T,n,3,2)) # (t,i,C_t,Z_{ti})

    for t in range(T):
        for i in range(n):
            for c_val in range(3):
                beliefs_Mti[t,i,c_val,:] = [psi_M[c_val,z] * delta_Lt_Mti_list[t,i,c_val] * delta_Nti_Mti_list[t,i,z] for z in [0,1]]

            # Normalizing
            beliefs_Mti[t,i,:,:] /= sum(sum(beliefs_Mti[t,i,:,:]))         #(t,i,C_t,Z_{ti})
    
    return beliefs_L_t, beliefs_Mti

In [ ]:
def p_z_t_i_cond_x(beliefs_M, t,i):
    return np.sum(beliefs_M[t-1,i-1,:,:], axis = 0)

def p_c_t_cond_x(beliefs_L,t):
    if t == T:
        return np.sum(beliefs_L[t-2,:,:], axis = 0)
    return np.sum(beliefs_L[t-1,:,:], axis = 1)

In [ ]:
def MAP(z_p):

    # Initialize 
    z_inf = 0
 
    # Traverse array elements from second
    # and compare every element with
    # current max
    for i in range(1, len(z_p)):
        if z_p[i] > z_p[z_inf]:
            z_inf = i
    return z_inf

In [6]:
def learn_par(C,Z,X, Print = False):
    n = X.shape[0]
    T = X.shape[1]

    # Estimating lambda's
    lambda1 = sum(sum((Z[:,t] * X[:,t] for t in range(T)))) / (sum(sum(Z)))
    lambda0 = sum(sum(((np.ones(shape = n)-Z[:,t]) * X[:,t] for t in range(T)))) / (n*T - sum(sum(Z)))

    # Indicator functions of C 
    C_0 = [C[t] == 0 for t in range(T)]
    C_1 = [C[t] == 1 for t in range(T)]
    C_2 = [C[t] == 2 for t in range(T)]

    # Estimating alpha_hat
    if (sum(C_1) > 0):
        sum1 = sum(sum(Z[i,t] * C_1[t] for i in range(n)) for t in range(T)) / (2 * sum(C_1) * n)
    else:
        sum1 = 0
    if (sum(C_0) > 0):
        sum2 = sum(sum((1-Z[i,t]) * C_0[t] for i in range(n)) for t in range(T)) / (2 * sum(C_0) * n)
    else:
        sum2 = 0
    alpha_hat = sum1 + sum2

    # Estimating beta hat
    beta_hat = sum((C_0[t+1] + C_1[t+1])*C_2[t] for t in range(T-1)) / (sum(C_2[0:T-1]))

    # Estimating gamma hat
    gamma_hat = sum((C_2[t+1])*(C_1[t]+C_0[t]) for t in range(T-1)) / (sum(C_1[0:T-1]+ C_0[0:T-1]))

    if (Print == True):
        print("lambda0_hat is: ", lambda0, "\nlambda1_hat is: ", lambda1, "\nalpha_hat is:", alpha_hat, "\nbeta_hat is: ", beta_hat, "\ngamme_hat is: ", gamma_hat)
    
    return lambda0 ,lambda1, alpha_hat, beta_hat, gamma_hat

In [ ]:
def EM(C, Z, X, alpha = 0.9, beta = 0.2, gamma = 0.1, lamb0=1, lamb1=5, N = 10):

    for i in range(N):

        T = Z.dim[0]
        n = Z.dim[1]

        # Inference - infer the most probable C,Z,X values

        beliefs_C, beliefs_Z = Clique_Beliefs(Xs = X, alpha = alpha_hat, beta = beta_hat , gamma = gamma_hat, lambda_p = [lambda0, lambda1])

        C_most_prob = [MAP(p_c_t_cond_x(beliefs_C,t+1)) for t in range(T)]
        Z_most_prob = np.empty(shape = (T,n))
        for t in range(T):
            Z_most_prob[t,:] = [MAP(p_z_t_i_cond_x(beliefs_Z,t+1,i+1)) for i in range(n)]

        # Learning - computing the most probable parameter estimates

        lambda0 ,lambda1, alpha_hat, beta_hat, gamma_hat = learn_par(C_most_prob, Z_most_prob, X, Print = False)    

    return par_est